In [18]:
import requests
import json
import os
import time
from tqdm import tqdm
import random

#API_KEY = "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"  # Jim
#API_KEY = "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"  # Jim2
#API_KEY = "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"  # Jim3
API_KEY = "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"  # Jim4
#API_KEY = "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"  # Lipu
#API_KEY = "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"  # Lipu2
#API_KEY = "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"  # Lipu3
#API_KEY = "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"  # Lipu4
#API_KEY = "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"  # Fahmida
#API_KEY = "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"  # Fahmida2
#API_KEY = "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"  # Fahmida3
#API_KEY = "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"  # Fahmida4
#API_KEY = "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"  # Fahmida4
#API_KEY = "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"  # Fahmida5
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "<YOUR_SITE_URL>",   # Optional
    "X-Title": "<YOUR_SITE_NAME>"        # Optional
}

# Base save directory
base_dir = "./generated/"

# Create category-specific directories
# os.makedirs(os.path.join(base_dir, "poems"), exist_ok=True)
# os.makedirs(os.path.join(base_dir, "novels"), exist_ok=True)
# os.makedirs(os.path.join(base_dir, "storys"), exist_ok=True)
# os.makedirs(os.path.join(base_dir, "essays"), exist_ok=True)

def load_counts():
    counts_file = os.path.join(base_dir, "counts.json")
    if os.path.exists(counts_file):
        with open(counts_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    else:
        return {"poem": 0, "novel": 0, "story": 0, "essay": 0}

def save_counts(counts):
    counts_file = os.path.join(base_dir, "counts.json")
    with open(counts_file, 'w', encoding='utf-8') as f:
        json.dump(counts, f, indent=4)

# Prompts per category
poem_prompts = [
    #"Write a bengali poem in 800-900 words",
    #"Write a bengali poem in 800-900 words",
    "যেকোনো বিষয়ে একটি বাংলা কবিতা লিখ (১৫০ শব্দের মধ্যে)",
    "ভ্রমণভিত্তিক একটি বাংলা কবিতা লিখ (১৫০ শব্দের মধ্যে)",
    "যেকোনো বিষয়ে একটি বাংলা কবিতা লিখ (১৫০ শব্দের মধ্যে)",    
    "দেশপ্রেম নিয়ে একটি বাংলা কবিতা লিখ (৭০০-৮০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা কবিতা লিখ (১৫০ শব্দের মধ্যে)",
    "মানবজীবন নিয়ে একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "বন্ধুত্ব নিয়ে একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "দৈনন্দিন জীবন নিয়ে ১৫০ শব্দের মধ্যে একটি বাংলা কবিতা লিখ",
    "একটি রোমান্টিক ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "শিশুদের জন্য একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ"
    "প্রযুক্তি নিয়ে একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "সাহস নিয়ে একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "প্রেম নিয়ে একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "প্রেম নিয়ে একটি ৭০০-৮০০ শব্দের বাংলা কবিতা লিখ",
    "তরুণদের নিয়ে একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "দৈনন্দিন জীবন নিয়ে একটি বাংলা কবিতা লিখ (১৫০ শব্দের মধ্যে)",
    "ন্যায়বিচার নিয়ে একটি ১৫০ শব্দের মধ্যে বাংলা কবিতা লিখ",
    "গ্রামীণ সৌন্দর্য নিয়ে একটি বাংলা কবিতা লিখ (১৫০ শব্দের মধ্যে)",
    "Write a bengali poem in 300-400 words",
    "Write a bengali poem in 300-400 words",
    "যেকোনো বিষয়ে একটি বাংলা কবিতা লিখ (১৫০ শব্দের মধ্যে)"


    
]

novel_prompts = [
    "যেকোনো বিষয়ে একটি বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "দৈনন্দিন জীবন নিয়ে একটি বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "দৈনন্দিন জীবন নিয়ে একটি বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "দৈনন্দিন জীবন নিয়ে একটি বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি মনস্তাত্ত্বিক বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি মনস্তাত্ত্বিক বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি মনস্তাত্ত্বিক বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি পারিবারিক বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি পারিবারিক বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি পারিবারিক বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা রহস্য উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা রহস্য উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা রহস্য উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা থ্রিলার উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা থ্রিলার উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা থ্রিলার উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)", 
    "সামাজিক বাস্তবতা নিয়ে একটি বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "সামাজিক বাস্তবতা নিয়ে একটি বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "সামাজিক বাস্তবতা নিয়ে একটি বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বাঙালির জাতিসত্তা নিয়ে একটি বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বাঙালির জাতিসত্তা নিয়ে একটি বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বাঙালির জাতিসত্তা নিয়ে একটি বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ভৌতিক বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ভৌতিক বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ভৌতিক বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বিজ্ঞান কল্পকাহিনি বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বিজ্ঞান কল্পকাহিনি বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বিজ্ঞান কল্পকাহিনি বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি রোমান্টিক বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি রোমান্টিক বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি রোমান্টিক বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি হাস্যরসাত্মক বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি হাস্যরসাত্মক বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি হাস্যরসাত্মক বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি দুঃখময় প্রেম নিয়ে বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি দুঃখময় প্রেম নিয়ে বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি দুঃখময় প্রেম নিয়ে বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ইতিহাসভিত্তিক বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ইতিহাসভিত্তিক বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ইতিহাসভিত্তিক বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি সামাজিক সমস্যা নিয়ে বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি সামাজিক সমস্যা নিয়ে বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি সামাজিক সমস্যা নিয়ে বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)"
    "নারীজীবন নিয়ে বাংলা উপন্যাসের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "নারীজীবন নিয়ে বাংলা উপন্যাসের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "নারীজীবন নিয়ে বাংলা উপন্যাসের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)"
    
]

story_prompts = [
    "যেকোনো বিষয়ে একটি বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "দৈনন্দিন জীবন নিয়ে একটি বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "দৈনন্দিন জীবন নিয়ে একটি বাংলা গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "দৈনন্দিন জীবন নিয়ে একটি বাংলা গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা ভৌতিক গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা ভৌতিক গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা ভৌতিক গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা ভ্রমণভিত্তিক গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা ভ্রমণভিত্তিক গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা ভ্রমণভিত্তিক গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা রোমান্টিক গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা রোমান্টিক গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা রোমান্টিক গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি দেশাত্মবোধক বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি দেশাত্মবোধক বাংলা গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি দেশাত্মবোধক বাংলা গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "গ্রামীণ জীবন নিয়ে একটি বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "গ্রামীণ জীবন নিয়ে একটি বাংলা গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "গ্রামীণ জীবন নিয়ে একটি বাংলা গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা হাস্যরসাত্মক গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা হাস্যরসাত্মক গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা হাস্যরসাত্মক গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)"
]


essay_prompts = [
    "যেকোনো বিষয়ে একটি বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "যেকোনো বিষয়ে একটি বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বিবিধ বিষয়ে একটি বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বিবিধ বিষয়ে একটি বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বিবিধ বিষয়ে একটি বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বাংলা ভাষা নিয়ে একটি প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"বাংলা ভাষা নিয়ে একটি প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"বাংলা ভাষা নিয়ে একটি প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "সামাজিক বাস্তবতা নিয়ে একটি প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"সামাজিক বাস্তবতা নিয়ে একটি প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"সামাজিক বাস্তবতা নিয়ে একটি প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি আত্মজৈবনিক বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি আত্মজৈবনিক বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি আত্মজৈবনিক বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "বাঙালির সংস্কৃতি নিয়ে একটি প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"বাঙালির সংস্কৃতি নিয়ে একটি প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"বাঙালির সংস্কৃতি নিয়ে একটি প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি শিক্ষামূলক বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ভ্রমণভিত্তিক বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি ভ্রমণভিত্তিক বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি ভ্রমণভিত্তিক বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ব্যঙ্গাত্মক বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ব্যঙ্গাত্মক বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি ব্যঙ্গাত্মক বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি নৈতিকমূলক বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি নৈতিকমূলক বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি নৈতিকমূলক বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি সাহিত্যভিত্তিক বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি সাহিত্যভিত্তিক বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি সাহিত্যভিত্তিক বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি দেশাত্মবোধক বাংলা প্রবন্ধের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি দেশাত্মবোধক বাংলা প্রবন্ধের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
	"একটি দেশাত্মবোধক বাংলা প্রবন্ধের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা হাস্যরসাত্মক গল্পের প্রথম অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা হাস্যরসাত্মক গল্পের মধ্য অংশ লিখ (৩০০-৪০০ শব্দ)",
    "একটি বাংলা হাস্যরসাত্মক গল্পের শেষ অংশ লিখ (৩০০-৪০০ শব্দ)"
]

# Text generation function
def generate_text(prompt):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers=HEADERS,
        data=json.dumps({
            "model": "x-ai/grok-code-fast-1",
            "messages": [{"role": "user", "content": prompt}]
        })
    )

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print("Error:", response.status_code, response.text)
        return None

# Modify the generate_samples function
def generate_samples(sample_type, prompts, count):
    save_dir = os.path.join(base_dir, f"{sample_type}s")
    os.makedirs(save_dir, exist_ok=True)
    
    # Load current counts
    counts = load_counts()
    current_count = counts[sample_type]
    
    for i in tqdm(range(count), desc=f"Generating {sample_type}s"):
        prompt = random.choice(prompts)
        text = generate_text(prompt)
        if text:
            # Increment count before saving
            current_count += 1
            file_path = os.path.join(save_dir, f"{sample_type}_{current_count}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text.strip())
            
            # Update counts in memory
            counts[sample_type] = current_count
            # Save counts after each successful generation
            save_counts(counts)
        else:
            time.sleep(5)
        time.sleep(1)

# Example usage:
generate_samples("poem", poem_prompts, 0)
generate_samples("novel", novel_prompts, 0)
generate_samples("story", story_prompts, 0)
generate_samples("essay", essay_prompts, 7)

Generating poems: 0it [00:00, ?it/s]
Generating novels: 0it [00:00, ?it/s]
Generating storys: 0it [00:00, ?it/s]
Generating essays: 100%|██████████| 7/7 [02:34<00:00, 22.02s/it]
